#Working on:


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#numpy
import numpy as np

#pandas
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

from IPython.display import Image
# plotting stuff
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
colorMap = sns.light_palette("blue", as_cmap=True)

#instal dabl
!pip install dabl > /dev/null


#istall datatable
!pip install datatable > /dev/null
import datatable as dt

# system
import warnings
warnings.filterwarnings('ignore')

# garbage collector to keep cleaning RAM
import gc

In [ ]:
%%time

dt_train = dt.fread('../input/jane-street-market-prediction/train.csv') 

In [ ]:
dt_train.head()

In [ ]:
%%time

df_train = dt_train.to_pandas()

In [ ]:
df_train.head()

In [ ]:
df_train.shape

In [ ]:
df_train.describe(include='all')

In [ ]:
len(df_train.date.unique())

From the above code block, training data has 500 unique days of trading events.

There are a total of 500 days of data in df_train.csv. Let us see at cumulative return over time

In [ ]:
#set x,y labels of plot facet.
fig, ax = plt.subplots(figsize=(15,5))
ax.set_xlabel("Trade", fontsize=18)
ax.set_ylabel("Cumulative daily return", fontsize=18)

#value to plot
cumsum_return = pd.Series(df_train['resp']).cumsum()
#plot
cumsum_return.plot()

#### investment Time Horizons
_An investment time horizon_ is the period where one expects to hold an investment for a specific goal e.g. riskier and less risky investment goals. The longer the time-horizon more riskier the investment. The shorter the time-horizon, less risky the investment.

In [ ]:
#plot 4 return i.e. resp_1, resp_2, resp_3, and resp_4

#set figure frame, labels, and facets.
fig, ax = plt.subplots(figsize=(15,5))
ax.set_xlabel("Trade", fontsize=18)
ax.set_ylabel("Day", fontsize=18)
ax.set_title("Cumulative return of resp and time-horizon for 500 days", fontsize=18)
plt.legend(loc="upper left")

#values to plot
cumsum_return = pd.Series(df_train['resp']).cumsum()
resp_1 = pd.Series(df_train['resp_1']).cumsum()
resp_2 = pd.Series(df_train['resp_2']).cumsum()
resp_3 = pd.Series(df_train['resp_3']).cumsum()
resp_4 = pd.Series(df_train['resp_4']).cumsum()
#plot
cumsum_return.plot()
resp_1.plot()
resp_2.plot()
resp_3.plot()
resp_4.plot()

In [ ]:
df_train.resp.hist(bins=2000)

In [ ]:
plt.figure(figsize = (12,5))
ax = sns.distplot(df_train['resp'], 
             bins=3000, 
             kde_kws={"clip":(-0.05,0.05)}, 
             hist_kws={"range":(-0.05,0.05)},
             color='blue',#darkcyan 
             kde=False);
plt.xlabel("Histogram of the resp values", size=18)

Let us calcualte `skewness` and `kurtosis`.

Definition:

A histogram is an effective graphical technique for showing both skewness and kurtosis of a data set[[1](https://www.itl.nist.gov/div898/handbook/eda/section3/eda35b.htm)].

A skewness is a measure of symmetry, or more precisely, the lack of symmetry. A data or distribution is symmetric if it looks the same to the left and right of its center point.

*Note: Skewness value nearing `0` shows normal distribution of data sets.*

Kurtosis on the other hand is a measure of whether the data are heavy tailed or light tailed relative to the normal distribution i.e. data sets with high kurtosis tend to have heavy tails or outliers. Data sets with low kurtosis tend to have light tails or lack of outliers. 

*Note: Kurtosis value around 3 indicate less heavy tail, and kurtosis value away from 3 shows, data set is heavily tailed.*

In [ ]:
#let check skewness and kurtosis of 'resp' column
print(f"SKEWNESS = {df_train['resp'].skew()}")
print(f"KURTOSIS = {df_train['resp'].kurtosis()}")

`resp` colum has heavy tails.

One needs to apply transformation to make the data normal or near normal because many classical statistical tests and models assume data normality i.e. normal distribution of data. In this respect, following techniques or transformations can be used:
* Box-Cox transformation,
* log transformation,
* square root transformation


Lets see distribution, skewness, and kurtosis of `weight` column.

In [ ]:
#figure
fig, ax = plt.subplots(figsize=(10,8))
ax.set_xlabel('Return weight', fontsize=18)
ax.set_ylabel('', fontsize=18)

weight_ls = pd.Series(df_train['weight'])
sns.distplot(weight_ls, bins=200, hist=False)
#sns.kdeplot(weight_ls)

plt.show()

In [ ]:
#let check skewness and kurtosis of 'weight' column
print(f"SKEWNESS = {df_train['weight'].skew()}")
print(f"KURTOSIS = {df_train['weight'].kurtosis()}")

Earning `weight` is right skewned and kurtosis value shows, it is heavily tailed because kurtosis value is greater than 3.

Approximately, 17% of the weights are `0`, indicating no earning. It would be intersting to see distribution of weight without those rows.

In [ ]:
#df_train[df_train['weight']>=0]['weight']
#df_train['weight']>=0
len(pd.Series(df_train['weight']!=0.0))


In [ ]:
#figure
fig, ax = plt.subplots(figsize=(10,8))
ax.set_xlabel('Return weight', fontsize=18)
ax.set_ylabel('', fontsize=18)

weight_ls = pd.Series(df_train[df_train['weight']>=0]['weight'])
sns.distplot(weight_ls, 
             bins=1400, 
             kde_kws={"clip":(0.001,1.4)}, 
             hist_kws={"range":(0.001,1.4)},
             color='blue', 
             kde=False)#sns.kdeplot(weight_ls)

plt.show()

Lets look at absolute earning by multiplying `weight` with `resp`.

In [ ]:
#creating a new variable in the data set `abs_earning`
#figure
fig, ax = plt.subplots(figsize=(10,8))
ax.set_xlabel('Absolute earning', fontsize=18)
ax.set_ylabel('', fontsize=18)

abs_earning = pd.Series(df_train['resp'] *df_train['weight'])
sns.distplot(abs_earning, bins=2000)
#sns.kdeplot(weight_ls)

plt.show()

In [ ]:
#figure
fig, ax = plt.subplots(figsize=(10,8))
ax.set_xlabel('Absolute earning = Weight * resp', fontsize=18)
ax.set_ylabel('', fontsize=18)

abs_earning = pd.Series(df_train['resp'] *df_train['weight'])
sns.distplot(abs_earning, 
             bins=1400, 
             kde_kws={"clip":(-0.05,0.05)}, 
             hist_kws={"range":(-0.05,0.05)},
             color='green', 
             kde=False)#sns.kdeplot(weight_ls)

plt.show()

**Reference:**

[1] Tutorial-on-reading-large-datasets(https://www.kaggle.com/rohanrao/tutorial-on-reading-large-datasets)

[2] Jane-street-eda-of-day-0-and-feature-importance(https://www.kaggle.com/carlmcbrideellis/jane-street-eda-of-day-0-and-feature-importance)